Outline
    
1. Import datasets

2. Data cleaning + export cleaned datasets to `datasets/cleaned` (Note: index column name should always be "airport_code")

    2.1 Clean `datasets/original/crash` (unfinished data1)
    
    2.2 Clean `datasets/original/delay/2009.csv`. Generate `unpleasant_2009.csv`
    
    2.3 Clean `datasets/original/airport/airport-extended.csv`. (city_name, long, lat, alt)
    
    2.4 Clean `datasets/original/city/uscities.csv`. Merge `us_cities_df` and `pleasant_2019_df`.
    

==========================================

In [96]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import matplotlib.pyplot as plt
import requests
from string import digits
import wget # you need to "pip install wget"  
import glob
import time

import plotly.graph_objects as go # use conda to install plotly --YD
import pandas as pd
import plotly.figure_factory as ff
import xml.etree.ElementTree as ET # to read one dataset in XML format

1. import datasets

From `A Pleasant Flight.ipynb` (now in the folder `previous_codes`)

In [97]:
crash_df = pd.read_csv("datasets/original/crash/Airplane_Crashes_and_Fatalities_Since_1908.csv") #data1
etree = ET.parse("datasets/original/crash/AviationData.xml") #data2
delay_2009_df = pd.read_csv("datasets/original/delay/2009.csv") #data3 (done)
    #need to download 2009.csv from https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

In [98]:
airport_loc_df = pd.read_csv("datasets/original/airport/airports-extended.csv") #data4
#https://www.kaggle.com/open-flights/airports-train-stations-and-ferry-terminals#airports-extended.csv

#aiport_code: city_name, latitude, longitude, altitude_ft
us_cities_df = pd.read_csv("datasets/original/city/uscities.csv") #data5

==========================================

2.1 Clean `datasets/original/crash`

clean data2

In [99]:
xml_root = etree.getroot()

interest_columns = ["EventId","EventDate","Location","Country","Latitude","Longitude","AirportCode","InjurySeverity","AircraftDamage",
                    "AircraftCategory","NumberOfEngines","EngineType","Schedule","TotalUninjured","TotalMinorInjuries",
                    "TotalSeriousInjuries","TotalFatalInjuries","WeatherCondition","BroadPhaseOfFlight","RegistrationNumber","PurposeOfFlight"]

NTSB_crash_df = pd.DataFrame(columns=interest_columns) # NTSB_crash_df: dataframe which collects airport information. -- YD 02/28/2020

In [100]:
for elem in xml_root: # This loop will run only once
    for row in elem: 
        if not(row.attrib["PurposeOfFlight"]=="Business"):  # Only care about business flights whose engine type is not 'Reciprocating'-- YD 02/28/2020
            continue
        if row.attrib["EngineType"]=="Reciprocating": 
            continue
        information = list()
        for interest in interest_columns:
            if not (row.attrib[interest]==""):
                information.append(row.attrib[interest])
            else:
                information.append(np.nan)
        row_information = pd.Series(information,index=interest_columns)
        NTSB_crash_df = NTSB_crash_df.append(row_information,ignore_index=True)
            
# this may need to run for a while. It takes 18 seconds on my computer
#     NTSB_crash_df

In [101]:
# NTSB_crash_df[(NTSB_crash_df["AircraftCategory"]=="Airplane") & (NTSB_crash_df["EngineType"]!="Reciprocating") & (NTSB_crash_df["Country"]=="United States")]

In [102]:
# (unfinished Crash Data Cleaning.ipynb)

==========================================

2.2 Clean `datasets/original/delay`. Generate `unpleasant_2009.csv`

clean data3

In [103]:
# DEPARTURE
unpleasant_2009_departure = pd.DataFrame()
unpleasant_2009_departure['total_departure'] = delay_2009_df.loc[:,["ORIGIN"]].groupby('ORIGIN').size()
unpleasant_2009_departure[["average_departure_delay","average_departure_taxi"]] = delay_2009_df.loc[:,["ORIGIN","DEP_DELAY","TAXI_OUT"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['average_departure_cancelled'] = delay_2009_df.loc[:,["ORIGIN","CANCELLED"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['averge_departure_distance'] = delay_2009_df.loc[:,["ORIGIN","DISTANCE"]].groupby('ORIGIN').mean()
    
#ARRIVAL
unpleasant_2009_arrival = pd.DataFrame()
unpleasant_2009_arrival['total_arrival'] = delay_2009_df.loc[:,["DEST"]].groupby('DEST').size()
unpleasant_2009_arrival[["average_arrival_delay","average_arrival_taxi"]] = delay_2009_df.loc[:,["DEST","ARR_DELAY","TAXI_IN"]].groupby('DEST').mean()
unpleasant_2009_arrival['average_arrival_diverted'] = delay_2009_df.loc[:,["DEST","DIVERTED"]].groupby('DEST').mean()
unpleasant_2009_arrival['averge_arrival_distance'] = delay_2009_df.loc[:,["DEST","DISTANCE"]].groupby('DEST').mean()
    
unpleasant_2009_departure['total_departure_lg10'] = unpleasant_2009_departure['total_departure'].apply(np.log10)
unpleasant_2009_arrival['total_arrival_lg10'] = unpleasant_2009_arrival['total_arrival'].apply(np.log10)

unpleasant_2009 = unpleasant_2009_departure.merge(unpleasant_2009_arrival,left_index=True,right_index=True)
unpleasant_2009.index.names = ["airport_code"]

In [119]:
unpleasant_2009

,total_departure,average_departure_delay,average_departure_taxi,average_departure_cancelled,averge_departure_distance,total_departure_lg10,total_arrival,average_arrival_delay,average_arrival_taxi,average_arrival_diverted,averge_arrival_distance,total_arrival_lg10
airport_code,,,,,,,,,,,,
ABE,4034,5.877463,13.556651,0.020327,562.069658,3.605736,4037,5.126924,4.179300,0.002973,561.860045,3.606059
ABI,2490,8.685573,8.305099,0.023293,158.000000,3.396199,2490,10.406404,3.518443,0.001606,158.000000,3.396199
ABQ,35582,5.050531,9.963724,0.005312,641.429965,4.551230,35577,2.210595,5.242186,0.000675,642.158726,4.551169
ABY,995,7.742564,10.823409,0.021106,146.000000,2.997823,997,6.965271,3.421859,0.002006,146.000000,2.998695
ACK,342,16.478788,20.081818,0.035088,199.000000,2.534026,343,2.516616,5.556886,0.008746,198.647230,2.535294
...,...,...,...,...,...,...,...,...,...,...,...,...
WRG,722,8.484979,9.507163,0.033241,61.569252,2.858537,722,10.140376,3.793054,0.016620,61.569252,2.858537
WYS,340,-6.779412,8.191176,0.000000,273.000000,2.531479,340,-3.958702,3.523529,0.002941,273.000000,2.531479
XNA,13755,7.796676,13.595218,0.029953,526.853435,4.138461,13764,6.735768,4.960809,0.005522,527.071563,4.138745


export `unpleasant_2009.csv`

In [105]:
unpleasant_2009.to_csv("datasets/cleaned/unpleasant_2009.csv")

==========================================

2.3 Clean `datasets/original/airport/airport-extended.csv`. Generate `airport_loc.csv`

In [106]:
#clean city_name
def clean_city_name(input_city):
    original = input_city
    input_city = str(input_city)
    input_city = input_city.strip()
    input_city = input_city.lower()
    
    input_city = input_city.replace(".","")
    input_city = input_city.replace("\\\\","")
    input_city = input_city.replace("-"," ")
    input_city = input_city.replace(" - "," ")
    input_city = input_city.replace("saint ","st")
    input_city = input_city.replace("east ","")
    input_city = input_city.replace("west ","")
    
    input_city = input_city.translate({ord(k): None for k in digits})
    
    if ('/' in input_city):
        input_city = input_city[:input_city.find('/')]
    if ('(' in input_city):
        input_city = input_city[:input_city.find('(')]
    if (',' in input_city):
        input_city = input_city[:input_city.find(',')]
    input_city = input_city.strip()   
    if (' ' in input_city):
        temp=input_city.find(' ')
        if (temp > 2):
            input_city = input_city[:input_city.find(' ')]
        else:
            if (input_city.find(' ',temp+1) != -1):
                input_city = input_city[temp+1:input_city.find(' ',temp+1)]
            else:
                input_city = input_city[temp+1:]
    input_city = input_city.strip()
    try:
        assert len(input_city) > 2
        assert input_city.replace(" ","").replace("'","").isalpha()
    except:
        print("This city name is prehaps incorrect: ",original,input_city,len(original))
    return input_city

In [107]:
airport_loc_df.columns = ["ID","name","city_name","country","airport_code","code4","latitude","longitude","altitude_ft","UTC_offset","DST","timezone","type","information_source"]
airport_loc_df = airport_loc_df.loc[:,["city_name","country","airport_code","latitude","longitude","altitude_ft"]]
airport_loc_df = airport_loc_df[airport_loc_df["country"]=="United States"]
airport_loc_df = airport_loc_df.loc[:,["city_name","airport_code","latitude","longitude","altitude_ft"]]
airport_loc_df = airport_loc_df[airport_loc_df["airport_code"]!="\\N"] # remove NAN in index
airport_loc_df = airport_loc_df.set_index("airport_code")

airport_loc_df["city_name"] = airport_loc_df["city_name"].apply(clean_city_name)

In [108]:
airport_loc_df

,city_name,latitude,longitude,altitude_ft
airport_code,,,,
BTI,barter,70.134003,-143.582001,2
K03,fort,70.613403,-159.860001,35
LUR,cape,68.875099,-166.110001,16
PIZ,point,69.732903,-163.005005,22
ITO,hilo,19.721399,-155.048004,38
...,...,...,...,...
XMR,cocoa,28.467600,-80.566597,10
ZZV,zanesville,39.944401,-81.892097,900
ENN,nenana,64.547302,-149.074005,362


In [109]:
#airport_loc_df.to_csv("datasets/cleaned/airport_loc.csv")

==========================================

2.4 Clean `datasets/original/city/uscities.csv`. Merge `us_cities_df` and `pleasant_2019_df`.

clean data5

In [110]:
us_cities_df = us_cities_df[["city","state_id","county_fips","county_name","population","density","lat","lng"]]
us_cities_df = us_cities_df.rename(columns = {"city":"city_name"})
us_cities_df["fips"] = us_cities_df["county_fips"]

def get_county_code(input_county):
    return int(input_county) % 1000

us_cities_df["county_fips"] = us_cities_df["county_fips"].apply(get_county_code)

us_cities_df["city_name"] = us_cities_df["city_name"].apply(clean_city_name)

This city name is prehaps incorrect:  Y-O Ranch o 9
This city name is prehaps incorrect:  Ho-Ho-Kus ho 9
This city name is prehaps incorrect:  St. Jo jo 6
This city name is prehaps incorrect:  Ty Ty ty 5
This city name is prehaps incorrect:  So-Hi hi 5
This city name is prehaps incorrect:  K. I. Sawyer i 12
This city name is prehaps incorrect:  G. L. García l 12


In [111]:
airport_prop_df = unpleasant_2009.merge(airport_loc_df,how='inner',left_index=True,right_index=True)

In [112]:
airport_code_df = airport_prop_df[[]]
#airport_code_df
airport_code_df.to_csv("datasets/cleaned/airtport_code.csv")

In [113]:
airport_prop_df

,total_departure,average_departure_delay,average_departure_taxi,average_departure_cancelled,averge_departure_distance,total_departure_lg10,total_arrival,average_arrival_delay,average_arrival_taxi,average_arrival_diverted,averge_arrival_distance,total_arrival_lg10,city_name,latitude,longitude,altitude_ft
airport_code,,,,,,,,,,,,,,,,
ABE,4034,5.877463,13.556651,0.020327,562.069658,3.605736,4037,5.126924,4.179300,0.002973,561.860045,3.606059,allentown,40.652100,-75.440804,393
ABI,2490,8.685573,8.305099,0.023293,158.000000,3.396199,2490,10.406404,3.518443,0.001606,158.000000,3.396199,abilene,32.411301,-99.681900,1791
ABQ,35582,5.050531,9.963724,0.005312,641.429965,4.551230,35577,2.210595,5.242186,0.000675,642.158726,4.551169,albuquerque,35.040199,-106.609001,5355
ABY,995,7.742564,10.823409,0.021106,146.000000,2.997823,997,6.965271,3.421859,0.002006,146.000000,2.998695,albany,31.535500,-84.194504,197
ACK,342,16.478788,20.081818,0.035088,199.000000,2.534026,343,2.516616,5.556886,0.008746,198.647230,2.535294,nantucket,41.253101,-70.060204,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRG,722,8.484979,9.507163,0.033241,61.569252,2.858537,722,10.140376,3.793054,0.016620,61.569252,2.858537,wrangell,56.484299,-132.369995,49
WYS,340,-6.779412,8.191176,0.000000,273.000000,2.531479,340,-3.958702,3.523529,0.002941,273.000000,2.531479,yellowstone,44.688400,-111.117996,6649
XNA,13755,7.796676,13.595218,0.029953,526.853435,4.138461,13764,6.735768,4.960809,0.005522,527.071563,4.138745,bentonville,36.281898,-94.306801,1287


In [114]:
airport_prop_df = airport_prop_df[["city_name", "latitude", "longitude"]]

In [118]:
airport_prop_df

,city_name,latitude,longitude
airport_code,,,
ABE,allentown,40.652100,-75.440804
ABI,abilene,32.411301,-99.681900
ABQ,albuquerque,35.040199,-106.609001
ABY,albany,31.535500,-84.194504
ACK,nantucket,41.253101,-70.060204
...,...,...,...
WRG,wrangell,56.484299,-132.369995
WYS,yellowstone,44.688400,-111.117996
XNA,bentonville,36.281898,-94.306801


In [115]:
city_climate_df = pd.DataFrame(columns=["code","population","density","avg_temp_sp","avg_temp_su","avg_temp_fa","avg_temp_wi","avg_precipitation_sp","avg_precipitation_su","avg_precipitation_fa","avg_precipitation_wi"])
city_search_df = pd.DataFrame(columns=["code","state_id","county_id","city_id","fips"])

In [116]:
for ind,row in airport_prop_df.iterrows():
    city = row["city_name"]
    target_lat = row["latitude"]
    target_lng = row["longitude"]
    try:      
        target_cities = us_cities_df[us_cities_df["city_name"]==city]
        
        if not (target_cities.shape[0] == 1):
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            target_cities.loc[:,"error"] = (target_cities.apply(calc_dis,axis=1))
            target_city = target_cities.sort_values(by="error").iloc[0]
            
            assert target_city["error"] < 1.5
            
            target_city = target_city.drop(["error"])
        elif (target_cities.shape[0] >= 1):
            target_city = target_cities.iloc[0]
        
        county = str(target_city["county_fips"])
        if (len(county)==1):
            county = "00" + county
        elif (len(county)==2):
            county = "0" + county

        city_search_df = city_search_df.append({"code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"]},ignore_index=True)        
    except:
        try:
            def calc_dis(input_):
                err = abs(target_lat - input_["lat"]) + abs(target_lng - input_["lng"])
                return err
            us_cities_df_copy = us_cities_df
            us_cities_df_copy.loc[:,"error"] = (us_cities_df.apply(calc_dis,axis=1))
            target_city = us_cities_df_copy.sort_values(by="error").iloc[0]
            assert target_city["error"] < 1.5
            county = str(target_city["county_fips"])
            if (len(county)==1):
                county = "00" + county
            elif (len(county)==2):
                county = "0" + county
            city_search_df = city_search_df.append({"code":ind,"state_id":target_city["state_id"],"county_id":county,"city_id":target_city.name,"fips":target_city["fips"]},ignore_index=True)
        except:
            print("No data for ",city)
            city_search_df = city_search_df.append({"code":ind,"state_id":np.nan,"county_id":np.nan,"city_id":np.nan,"fips":np.nan},ignore_index=True)

#special case for DC
for ind,row in city_search_df.iterrows():
    if (row["code"]=="DCA"):
        city_search_df.iloc[ind]["state_id"]="MD"
        city_search_df.iloc[ind]["county_id"]="511"

        
        
city_search_df = city_search_df[(city_search_df["state_id"]!="HI") & (city_search_df["state_id"]!="AK")]
        
#city_search_df.head()

In [117]:
city_search_df

,code,state_id,county_id,city_id,fips
0,ABE,PA,077,10988,42077
1,ABI,TX,441,5333,48441
2,ABQ,NM,001,3742,35001
3,ABY,GA,095,17309,13095
4,ACK,MA,019,19193,25019
...,...,...,...,...,...
284,VLD,GA,185,17227,13185
285,VPS,FL,091,2510,12091
287,WYS,MT,031,26356,30031
288,XNA,AR,007,15451,5007
